In [46]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import datetime
import pandas as pd
import lxml

In [47]:
#gets alt tags for images that are otherwise given in table as NaN, for levels and country
def getAlt(x):
    for img in x.find_all('img'):
        if(img.get("alt")):
#             print(img.get("alt"))
            img.string = img.get("alt")

#copied code from scrape_mars.py
def getLeaderboardData():
    url = "https://app.codesignal.com/leaderboard/"

    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)

    browser.visit(url)
    
    #add a delay so page fully loads
    time.sleep(6)
    
    #set leaderboard to all-time
    browser.find_by_css('div.-start').first.click()
    
    browser.find_by_css("div#select-1-option-total").first.click()
    
    time.sleep(4)
    
    #close cookie saver notification banner
    browser.find_by_css('div.close-banner').last.click()
    
    #start collecting data
    
    for i in range(100):
        
        try:
            current_leaderboard = browser.html

            news_soup = bs(current_leaderboard, "html.parser")

            top_scorers = news_soup.find_all("table")[0]
#             print(top_scorers.prettify())
            getAlt(top_scorers)

            if(i == 0):
                output = pd.read_html(top_scorers.prettify(), header=0, flavor="bs4")[0]
            else:
                output = output.append(pd.read_html(top_scorers.prettify(), header=0, flavor="bs4")[0], ignore_index=True)

            browser.find_by_css('div.icon.-size-12').last.click()
        except:
            print("Missing page #{} of 100".format(i))
        
        time.sleep(8)

    browser.quit()

    return output

In [48]:
output_df = getLeaderboardData()
output_df

,#,CodeSignaler,Level,Country,XP
0,1,lonerz,Level 100,Holy See,4553861
1,2,AWice,Level 100,Canada,3678454
2,3,romatua_l,Level 100,Indonesia,3573800
3,4,flatorez,Level 100,Czech Republic,3429424
4,5,pythagoras_i,Level 100,Germany,3247495
...,...,...,...,...,...
995,996,santosh_savan,Level 69,India,222196
996,997,tsvetoslav_t,Level 69,Bulgaria,222133
997,998,stephentran,Level 69,Vietnam,222069
998,999,artskeem,Level 69,United States,221946


In [49]:
len(output_df)

1000

In [50]:
output_df.to_csv("./output/leaderboard_{}.csv".format(datetime.date.today()), index=False)